In [1]:
import oracledb
import pandas as pd


In [2]:
# Replace with your actual credentials
conn = oracledb.connect(
    user="system",
    password="draxthemup7",
    dsn="localhost/XEPDB1"  # Adjust based on your Oracle setup
)

cursor = conn.cursor()
print("Connected to Oracle DB!")


Connected to Oracle DB!


In [3]:
df = pd.read_csv('../data/processed/bank_review_sentiment_theme.csv')

# Confirm it loaded
df.head()


,short_review,cleaned_review,sentiment,sentiment_score,theme,bank,rating,date,source
0,"""Why don’t your ATMs support account-to-accoun...",atms support account account transfer like cou...,NEGATIVE,0.996465,Transaction Performance,CBE,4,2025-06-06,Google Play
1,what is this app problem???,app problem,NEGATIVE,0.999623,Other,CBE,1,2025-06-05,Google Play
2,the app is proactive and a good connections.,app proactive good connection,POSITIVE,0.999868,Other,CBE,5,2025-06-05,Google Play
3,I cannot send to cbebirr app. through this app.,send cbebirr app app,NEGATIVE,0.995335,Other,CBE,3,2025-06-05,Google Play
4,good,good,POSITIVE,0.999816,Other,CBE,4,2025-06-05,Google Play


In [4]:
cursor.execute("""
    BEGIN
        EXECUTE IMMEDIATE 'DROP TABLE REVIEW_BANKS';
    EXCEPTION
        WHEN OTHERS THEN
            IF SQLCODE != -942 THEN
                RAISE;
            END IF;
    END;
""")

cursor.execute("""
    CREATE TABLE REVIEW_BANKS (
        ReviewID NUMBER GENERATED BY DEFAULT AS IDENTITY PRIMARY KEY,
        Short_Review VARCHAR2(1000),
        Cleaned_Review VARCHAR2(2000),
        Sentiment VARCHAR2(20),
        Sentiment_Score NUMBER(5,2),
        Theme VARCHAR2(100),
        Bank VARCHAR2(100),
        Rating NUMBER(2,1),
        Review_Date DATE,
        Source VARCHAR2(100)
    )
""")

print("Table REVIEW_BANKS created!")


Table REVIEW_BANKS created!


In [7]:
for idx, row in df.iterrows():
    try:
        # Safe parsing of numerical fields
        sentiment_score = float(row['sentiment_score']) if pd.notnull(row['sentiment_score']) else None
        rating = float(row['rating']) if pd.notnull(row['rating']) else None

        # Safe date parsing
        review_date = pd.to_datetime(row['date'], errors='coerce')
        if pd.isnull(review_date):
            review_date = None
        else:
            review_date = review_date.date()

        cursor.execute("""
            INSERT INTO REVIEW_BANKS (
                Short_Review, Cleaned_Review, Sentiment,
                Sentiment_Score, Theme, Bank,
                Rating, Review_Date, Source
            ) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9)
        """, (
            row['short_review'],
            row['cleaned_review'],
            row['sentiment'],
            sentiment_score,
            row['theme'],
            row['bank'],
            rating,
            review_date,
            row['source']
        ))

    except Exception as e:
        print(f"Error on row {idx}: {e}")

conn.commit()
print("Data inserted successfully!")


Error on row 266: DPY-4004: invalid number
Error on row 429: DPY-4004: invalid number
Error on row 612: DPY-4004: invalid number
Error on row 892: DPY-4004: invalid number
Error on row 990: DPY-4004: invalid number
Error on row 1156: DPY-4004: invalid number
Error on row 1170: DPY-4004: invalid number
Error on row 1231: DPY-4004: invalid number
Error on row 1302: DPY-4004: invalid number
Error on row 1364: DPY-4004: invalid number
Error on row 1401: DPY-4004: invalid number
Error on row 1416: DPY-4004: invalid number
Data inserted successfully!


In [8]:
query = "SELECT * FROM REVIEW_BANKS WHERE ROWNUM <= 10"
cursor.execute(query)
rows = cursor.fetchall()

columns = [col[0] for col in cursor.description]
df_from_db = pd.DataFrame(rows, columns=columns)

df_from_db


,REVIEWID,SHORT_REVIEW,CLEANED_REVIEW,SENTIMENT,SENTIMENT_SCORE,THEME,BANK,RATING,REVIEW_DATE,SOURCE
0,1,"""Why don’t your ATMs support account-to-accoun...",atms support account account transfer like cou...,NEGATIVE,1.00,Transaction Performance,CBE,4.0,2025-06-06,Google Play
1,2,what is this app problem???,app problem,NEGATIVE,1.00,Other,CBE,1.0,2025-06-05,Google Play
2,3,the app is proactive and a good connections.,app proactive good connection,POSITIVE,1.00,Other,CBE,5.0,2025-06-05,Google Play
3,4,I cannot send to cbebirr app. through this app.,send cbebirr app app,NEGATIVE,1.00,Other,CBE,3.0,2025-06-05,Google Play
4,5,good,good,POSITIVE,1.00,Other,CBE,4.0,2025-06-05,Google Play
5,6,not functional,functional,NEGATIVE,1.00,Other,CBE,1.0,2025-06-05,Google Play
6,7,everytime you uninstall the app you have to re...,everytime uninstall app reach physically oldy ...,NEGATIVE,0.99,Other,CBE,1.0,2025-06-04,Google Play
7,8,The name of our account is the name of our acc...,account account apple account reject apple acc...,NEGATIVE,1.00,Other,CBE,4.0,2025-06-04,Google Play
8,9,best,well,POSITIVE,1.00,Other,CBE,5.0,2025-06-04,Google Play
9,10,Bezabih,bezabih,POSITIVE,0.97,Other,CBE,5.0,2025-06-04,Google Play


In [9]:
errors = []

for idx, row in df.iterrows():
    try:
        # [same as above]
        ...
    except Exception as e:
        errors.append((idx, str(e)))

print("Failed Rows:", errors)


Failed Rows: []


In [14]:
import pandas as pd
import oracledb

# Connect to your Oracle DB (adjust these to your setup)
conn = oracledb.connect(
    user="system",
    password="draxthemup7",
    dsn="localhost/XEPDB1"  # Adjust based on your Oracle setup
)

# SQL query to get all rows
query = "SELECT * FROM REVIEW_BANKS"

# Use pandas to load into a DataFrame
df_from_db = pd.read_sql(query, con=conn)

# Show the full DataFrame
pd.set_option('display.max_rows', None)   # show all rows
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.width', 1000)       # widen display

df_from_db.head(1430)  # or just df_from_db to see everything


C:\Users\Bob\AppData\Local\Temp\ipykernel_7176\2442609301.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_from_db = pd.read_sql(query, con=conn)


,REVIEWID,SHORT_REVIEW,CLEANED_REVIEW,SENTIMENT,SENTIMENT_SCORE,THEME,BANK,RATING,REVIEW_DATE,SOURCE
0,1,"""Why don’t your ATMs support account-to-accoun...",atms support account account transfer like cou...,NEGATIVE,1.00,Transaction Performance,CBE,4.0,2025-06-06,Google Play
1,2,what is this app problem???,app problem,NEGATIVE,1.00,Other,CBE,1.0,2025-06-05,Google Play
2,3,the app is proactive and a good connections.,app proactive good connection,POSITIVE,1.00,Other,CBE,5.0,2025-06-05,Google Play
3,4,I cannot send to cbebirr app. through this app.,send cbebirr app app,NEGATIVE,1.00,Other,CBE,3.0,2025-06-05,Google Play
4,5,good,good,POSITIVE,1.00,Other,CBE,4.0,2025-06-05,Google Play
5,6,not functional,functional,NEGATIVE,1.00,Other,CBE,1.0,2025-06-05,Google Play
6,7,everytime you uninstall the app you have to re...,everytime uninstall app reach physically oldy ...,NEGATIVE,0.99,Other,CBE,1.0,2025-06-04,Google Play
7,8,The name of our account is the name of our acc...,account account apple account reject apple acc...,NEGATIVE,1.00,Other,CBE,4.0,2025-06-04,Google Play
8,9,best,well,POSITIVE,1.00,Other,CBE,5.0,2025-06-04,Google Play
9,10,Bezabih,bezabih,POSITIVE,0.97,Other,CBE,5.0,2025-06-04,Google Play
